<a href="https://colab.research.google.com/github/rafa-cc/Proyecto-Final-R/blob/main/Proyecto_Final_R_Emp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
import pandas as pd
import numpy as np
import statistics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go
from plotly.io import write_image

# __Código de thema para gráficos__

In [ ]:
# Definir un tema personalizado para el gráfico en Plotly
tam_titulo = 22
tam_ejes = 14
tam_tics_legend = 10
gris_600 = '#272727'
gris_500 = '#595959'
gris_100 = '#F5F5F5'

my_custom_theme = {
    'layout': {
        'title_font': {
            'family': 'Inter semibold',
            'size': tam_titulo,
            'color': gris_600
        },
        'title_x': 0.075,  # Centrar el título horizontalmente
        'title_y': 0.925,
        'xaxis': {
            'showgrid' : True, 'gridwidth' : 2, 'gridcolor' : 'White',
            'title_font': {
                'family': 'Inter semibold',
                'size': tam_ejes,
                'color': gris_600
            },
            'tickfont': {
                'family': 'Raleway',
                'size': tam_tics_legend,
                'color': gris_500
            },
            'title_standoff': 5
        },
        'yaxis': {
            'showgrid' : True, 'gridwidth' : 2, 'gridcolor' : 'White',
            'title_font': {
                'family': 'Inter semibold',
                'size': tam_ejes,
                'color': gris_600
            },
            'tickfont': {
                'family': 'Raleway',
                'size': tam_tics_legend,
                'color': gris_500
            },
            'title_standoff': 5
        },
        'legend': {
            'font': {
                'family': 'Raleway',
                'size': tam_tics_legend,
                'color': gris_500
            },
            'title_font': {
                'family': 'Raleway',
                'size': tam_tics_legend,
                'color': gris_500
            },
            #'y': 1.02,  # Ajustar la posición vertical de la leyenda
            #'x': 0.08,   # Centrar la leyenda horizontalmente
            #'xanchor': 'center',  # Anclaje horizontal al centro
            #'yanchor': 'bottom',
            'bgcolor': gris_100,
            'tracegroupgap': 20
        },
        'width': 21 * 37.7952756,  # Convertir de cm a píxeles (1 cm = 37.7952756 píxeles)
        'height': 13 * 37.7952756,
        'margin': {'l': 1 * 37.7952756, 'r': 1.5 * 37.7952756, 't': 2.5 * 37.7952756, 'b': 1 * 37.7952756},
        'plot_bgcolor': gris_100,
    }
}

# __Clasificación de los datos__

In [ ]:
# Cargar los datos guardados en el análisis hecho con R de la sección _Análisis de empaquetado_
df_py = pd.read_csv('https://raw.githubusercontent.com/rafa-cc/Proyecto-Final-R/main/data_emp.csv')
df_py.head()

,Length,Width,Height
0,0.1,5.1,6.2
1,0.2,4.4,6.8
2,0.3,0.3,0.7
3,0.3,0.3,0.7
4,0.3,2.9,4.3


Los datos se normalizan respecto a la mediana pues al final lo que caracteriza a cada forma es la distancia de los valores extremos respecto a ese punto medio, por ello fue importane ordenar por filas de menor a mayor cuando se manipuló con `R`

In [ ]:
def normal(triada):
    # Calcular la mediana de la triada
    med = statistics.median(triada)

    # Normalizar cada número en la triada respecto a la mediana. Eliminar el segundo elemento pues siempre será 1
    triada_normalizada = [round(x / med, 2) for i, x in enumerate(triada) if i != 1]

    return triada_normalizada

In [ ]:
# Crear una columna con una lista de las dimensiones normalizadas
df_py['normal'] = df_py.apply(normal, axis=1)
df_py.head(n=2)

,Length,Width,Height,normal
0,0.1,5.1,6.2,"[0.02, 1.22]"
1,0.2,4.4,6.8,"[0.05, 1.55]"


El modelo se entrena con los valores posibles que pueden tener las formas de los objetos. _Por ejemplo un cubo puede tener su mínimo entre $\sf[0.66,1]$ y su máximo entre $\sf[1,1.33]$_. Entonces se colocan algunas combinaciones para que el modelo considere ese rango de opciones.

In [ ]:
# Ejemplo de datos etiquetados para entrenar el modelo
data = [
    ((0.64, 1.0), 'Cubo'),
    ((1.0, 1.0), 'Cubo'),
    ((1.0, 1.33), 'Cubo'),
    ((0.64, 1.33), 'Cubo'),
    ((0.88, 1.0), 'Cubo'),
    ((1.0, 1.0), 'Cubo'),
    ((1.0, 1.15), 'Cubo'),
    ((0.88, 1.15), 'Cubo'),
    ((0.88, 1.33), 'Cubo'),
    ((0.64, 1.11), 'Cubo'),
    ((0.7, 1.2), 'Cubo'),

    ((0.11, 1.0), 'Pizza'),
    ((0.33, 1.0), 'Pizza'),
    ((0.4, 1.0), 'Pizza'),
    ((0.11, 1.33), 'Pizza'),
    ((0.33, 1.33), 'Pizza'),
    ((0.4, 1.33), 'Pizza'),
    ((0.4, 1.11), 'Pizza'),
    ((0.33, 1.11), 'Pizza'),
    ((0.11, 1.11), 'Pizza'),
    ((0.45, 1.0), 'Pizza'),
    ((0.45, 1.11), 'Pizza'),

    ((1.0, 2.0), 'Leche'),
    ((1.0, 1.66), 'Leche'),
    ((1.0, 1.4), 'Leche'),
    ((0.88,2.0), 'Leche'),
    ((0.88,1.66), 'Leche'),
    ((0.88,1.4), 'Leche'),
    ((0.66,2.0), 'Leche'),
    ((0.66,1.66), 'Leche'),
    ((0.66,1.4), 'Leche'),

    ((0.5,2.11), 'Estaca'),
    ((0.5,2.66), 'Estaca'),
    ((0.5,2.4), 'Estaca'),
    ((1.0,2.11), 'Estaca'),
    ((1.0,2.66), 'Estaca'),
    ((1.0,2.4), 'Estaca'),
    ((1.0,2.88), 'Estaca'),
    ((1.0,3), 'Estaca'),
    ((1.0,4), 'Estaca'),
    ((0.88, 2.0), 'Estaca'),
    ((0.88, 2.66), 'Estaca'),
    ((0.88, 2.4), 'Estaca'),
    ((0.66, 2.0), 'Estaca'),
    ((0.66, 2.66), 'Estaca'),
    ((0.66, 2.4), 'Estaca'),

    ((0.11,1.4), 'Tabla'),
    ((0.11,1.6), 'Tabla'),
    ((0.11,1.8), 'Tabla'),
    ((0.11,2.0), 'Tabla'),
    ((0.11,2.4), 'Tabla'),
    ((0.11,2.6), 'Tabla'),
    ((0.33,1.4), 'Tabla'),
    ((0.33,1.6), 'Tabla'),
    ((0.33,1.8), 'Tabla'),
    ((0.33,2.0), 'Tabla'),
    ((0.33,2.4), 'Tabla'),
    ((0.33,2.6), 'Tabla'),
    ((0.4,1.4), 'Tabla'),
    ((0.4,1.6), 'Tabla'),
    ((0.4,1.8), 'Tabla'),
    ((0.4,2.0), 'Tabla'),
    ((0.4,2.4), 'Tabla'),

    ((0.6, 1.2), 'Desco'),
    ((0.6, 1.33), 'Desco'),
    ((0.6, 1.4), 'Desco'),
    ((0.6, 1.6), 'Desco'),
    ((0.6, 2.0), 'Desco'),
    ((0.55,1.0), 'Desco'),
    ((0.55,1.2), 'Desco'),
    ((0.55,1.33), 'Desco'),
    ((0.55,1.4), 'Desco'),
    ((0.55,1.4), 'Desco'),
    ((0.55,1.6), 'Desco'),
    ((0.55,2.0), 'Desco'),
]

# Separar triadas y etiquetas
X = [entry[0] for entry in data]
y = [entry[1] for entry in data]

# Dividir el conjunto de datos de manera 'aleatoria' (se usó la misma semilla en R)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=127)

# Crear y entrenar el modelo
model = SVC()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.7333333333333333


In [ ]:
# Crear una función para aplicar el modelo a cada triada
def clasificar_triada(triada):
    etiqueta = model.predict([triada])[0]
    return etiqueta

# Aplicar la función a la columna 'normal' y crear una nueva columna 'etiqueta'
df_py['etiqueta'] = df_py['normal'].apply(clasificar_triada)

# __Comparación de Clasificaciones__

In [ ]:
# Todo el proceso anterior se hizo tambien pero con R
df_r = pd.read_csv('https://raw.githubusercontent.com/rafa-cc/Proyecto-Final-R/main/data__emp_class_R.csv')

Se calcula la distancia al origen pues, si se considera el origen como una esquina de la caja, el punto será la esquina opuesta y esto es directamente proporcional al volumen de la caja (_entre más se aleja el punto de la caja, más grande es ésta_)

In [ ]:
# Crear la columna 'distance' las cuales son la distancia al origen de cada punto
df_py['distance'] = round(np.sqrt(df_py['Length']**2 + df_py['Width']**2 + df_py['Height']**2),2)

# Filtrar los puntos con ditancia menor o igual a 21 pues ahi se encuentra el 95% de los datos
df_temp = df_py.query('distance <= 21')

In [ ]:
# Crear la columna 'distance' las cuales son la distancia al origen de cada punto
df_r['distance'] = round(np.sqrt(df_r['Length']**2 + df_r['Width']**2 + df_r['Height']**2),2)

# Filtrar los puntos con ditancia menor o igual a 21 pues ahi se encuentra el 95% de los datos
df_temp_2 = df_r.query('distance <= 21')

Comprobación de que el cuantil 95 corresponde a la distancia 21

In [ ]:
print(round(df_py['distance'].quantile(0.95),1))

fig = go.Figure()

fig.add_trace(go.Histogram(x=df_py['distance'],marker_color='#7D58EB'))

# Líneas verticales para la media
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df_py['distance'].mean(),
        x1=df_py['distance'].mean(),
        y0=0,
        y1=178,
        line=dict(color="#E71224", width=2.5),
    )
)

# Línea punteada que representa el cuantil 95
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df_py['distance'].quantile(0.95),
        x1=df_py['distance'].quantile(0.95),
        y0=0,
        y1=178,
        line=dict(color="#FFC114", width=2.5, dash="dash"),
    )
)

fig.update_layout(
    title='Histograma de Distribución <br>de la Distancia al Origen',
    xaxis=dict(title='Cantidad de datos'),
    yaxis=dict(title='Frecuencia'),
    width= 21 * 37.7952756, height= 13 * 37.7952756,
    margin={'l': 1 * 37.7952756, 'r': 1.5 * 37.7952756, 't': 2.5 * 37.7952756, 'b': 1 * 37.7952756}
)
fig.update_layout(template=my_custom_theme)

fig.show()

20.5


Ahora se puede ver cómo los datos fueron categorizados y cada etiqueta forma una especie de línea.

> 💡 Notar como el conjunto de datos parece una pirámide.

In [ ]:
fig = go.Figure()

for cluster in df_temp['etiqueta'].unique():
    df_cluster = df_temp[df_temp['etiqueta'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Cluster {cluster}',
        hovertemplate='Length: %{x}<br>Width: %{y}<br>Height: %{z}'
    ))

fig.update_layout(title='Datos Categorizados con Python',
    scene=dict(
      xaxis_title='Length',
      yaxis_title='Width',
      zaxis_title='Height',
      xaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      yaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      zaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
))
fig.update_layout(template=my_custom_theme)

fig.show()

In [ ]:
# Guardar la figura como archivo HTML interactivo
fig.write_html('ctg_py.html')

Se recategorizan los datos que calculó mal el modelo (_categorias **Pizza** y **cubo**_)

In [ ]:
# Los datos que estén a la izquierda el plano y = 5x - 6 y sean categoria 'Pizza' se reasignan los valores
indices = df_temp.loc[(df_temp['etiqueta'] == 'Pizza') & (((df_temp['Width'] + 6)/df_temp['Length']) < 5)].index
df_temp.loc[indices, 'etiqueta'] = 'Desco'

# Asignar los empaques pequeños directamente en la categoria 'Cubo'
indices = df_temp.loc[(df_temp['distance'] <= 4)].index
df_temp.loc[indices, 'etiqueta'] = 'Cubo'

In [ ]:
# Los datos que estén a la izquierda el plano y = 1.37x y sean categoria 'Desco' se reasignan los valores
indices = df_temp.loc[(df_temp['etiqueta'] == 'Desco') & ((df_temp['Width']/df_temp['Length']) < 1.37)].index
df_temp.loc[indices, 'etiqueta'] = 'Cubo'

In [ ]:
fig = go.Figure()

for cluster in df_temp['etiqueta'].unique():
    df_cluster = df_temp[df_temp['etiqueta'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Cluster {cluster}',
        hovertemplate='Length: %{x}<br>Width: %{y}<br>Height: %{z}'
    ))

fig.update_layout(title='Datos Recategorizados con Python',
    scene=dict(
      xaxis_title='Length',
      yaxis_title='Width',
      zaxis_title='Height',
      xaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      yaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      zaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
))
fig.update_layout(template=my_custom_theme)

fig.show()

In [ ]:
# Guardar la figura como archivo HTML interactivo
fig.write_html('rectg_py.html')

In [ ]:
df_temp.head()

,Length,Width,Height,normal,etiqueta,distance
0,0.1,5.1,6.2,"[0.02, 1.22]",Pizza,8.03
1,0.2,4.4,6.8,"[0.05, 1.55]",Tabla,8.10
2,0.3,0.3,0.7,"[1.0, 2.33]",Cubo,0.82
3,0.3,0.3,0.7,"[1.0, 2.33]",Cubo,0.82
4,0.3,2.9,4.3,"[0.1, 1.48]",Tabla,5.20


In [ ]:
# Eliminar la columna 'normal' porque si no el csv no se genera
df_temp = df_temp.drop('normal', axis=1)
# Guardar el dataframe para regresar al análisis con R
df_temp.to_csv('data_emp_class.csv',index=False)

In [ ]:
fig = go.Figure()

for cluster in df_temp_2['etiqueta'].unique():
    df_cluster = df_temp_2[df_temp_2['etiqueta'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Cluster {cluster}',
        hovertemplate='Length: %{x}<br>Width: %{y}<br>Height: %{z}'
    ))

fig.update_layout(title='Datos Categorizados con R',
    scene=dict(
      xaxis_title='Length',
      yaxis_title='Width',
      zaxis_title='Height',
      xaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      yaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      zaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
))
fig.update_layout(template=my_custom_theme)

fig.show()

In [ ]:
# Guardar la figura como archivo HTML interactivo
fig.write_html('ctg_r.html')

Se recategorizan los datos que calculó mal el modelo (_categoria **Tabla**_)

In [ ]:
df_temp_2.head()

,Length,Width,Height,etiqueta,distance
0,0.1,5.1,6.2,Pizza,8.03
1,0.2,4.4,6.8,Tabla,8.10
2,0.3,0.3,0.7,Estaca,0.82
3,0.3,0.3,0.7,Estaca,0.82
4,0.3,2.9,4.3,Tabla,5.20


In [ ]:
# Los datps que estén arriba del plano z = 4.5y - 1 y sean categoria 'Tabla' se reasignan los valores
indices = df_temp_2.loc[(df_temp_2['etiqueta'] == 'Tabla') & (((df_temp_2['Height'] + 1)/df_temp_2['Width']) > 4)].index
df_temp_2.loc[indices, 'etiqueta'] = 'Estaca'

# Asignar los empaques pequeños directamente en la categoria 'Cubo'
indices = df_temp_2.loc[(df_temp_2['distance'] <= 4)].index
df_temp_2.loc[indices, 'etiqueta'] = 'Cubo'

In [ ]:
fig = go.Figure()

for cluster in df_temp_2['etiqueta'].unique():
    df_cluster = df_temp_2[df_temp_2['etiqueta'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Cluster {cluster}',
        hovertemplate='Length: %{x}<br>Width: %{y}<br>Height: %{z}'
    ))

fig.update_layout(title='Datos Recategorizados con R',
    scene=dict(
      xaxis_title='Length',
      yaxis_title='Width',
      zaxis_title='Height',
      xaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      yaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      zaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
))
fig.update_layout(template=my_custom_theme)

fig.show()

In [ ]:
# Guardar la figura como archivo HTML interactivo
fig.write_html('rctg_r.html')

In [ ]:
# Guardar el dataframe para regresar al análisis con R
df_temp_2.to_csv('data_emp_reclass_R.csv',index=False)

# __Medidas de las Cajas__

In [ ]:
df_temp_2 = df_temp_2[df_temp_2['etiqueta']=='Leche']

In [ ]:
from sklearn.cluster import KMeans
# Definir el número de clusters
k = 5

# Crear el modelo KMeans
kmeans = KMeans(n_clusters=k)

# Ajustar el modelo a los datos
kmeans.fit(df_temp_2[['Length', 'Width', 'Height']])

# Agregar las etiquetas de los clusters al DataFrame
df_temp_2['cluster'] = kmeans.labels_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [ ]:
fig = go.Figure()

for cluster in df_temp_2['cluster'].unique():
    df_cluster = df_temp_2[df_temp_2['cluster'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Distancia {cluster}',
        hovertemplate='Length: %{x}<br>Width: %{y}<br>Height: %{z}'
    ))

fig.update_layout(title='Clusters de la Categoría -Leche-',
    scene=dict(
      xaxis_title='Length',
      yaxis_title='Width',
      zaxis_title='Height',
      xaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      yaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
      zaxis=dict(backgroundcolor=gris_100, color=gris_500,title_font=dict(family='Raleway', size=12),tickfont=dict(family='Raleway', size=12)),
))
fig.update_layout(template=my_custom_theme)

fig.show()

In [ ]:
# Guardar la figura como archivo HTML interactivo
fig.write_html('cltr_.html')